## 第一部分：复现课堂代码

### 一、 Dynamic Programming

In [1]:
from collections import defaultdict

In [2]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]

price = defaultdict(int)
for i,p in enumerate(original_price):
    price[i+1] = p
    
print(price)

defaultdict(<class 'int'>, {1: 1, 2: 5, 3: 8, 4: 9, 5: 10, 6: 17, 7: 17, 8: 20, 9: 24, 10: 30, 11: 35})


In [3]:
def r(n):
    return max([price[n]] + [r(i) + r(n-i) for i in range(1,n)])

In [4]:
r(10)

30

In [5]:
r(11)

35

In [6]:
from functools import wraps

In [7]:
def memo(f): 
    memo.already_computed = {}
    
    @wraps(f)
    def _wrap(arg):
        result = None
        if arg in memo.already_computed: 
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    
    return _wrap

In [8]:
solution = {}

@memo
def r(n):
    max_price, max_split = max([(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0])
    solution[n] = (n - max_split, max_split) 
    return max_price

In [9]:
r(15)

45

In [10]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2)}

In [11]:
def parse_solution(n):
    left_split,right_split = solution[n]
    if right_split == 0:return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [12]:
r(30)

92

In [13]:
parse_solution(30)

[11, 11, 6, 2]

### 二、Edit Distance

In [14]:
from functools import lru_cache

In [15]:
solution = {}

In [16]:
@lru_cache(maxsize=2**10)
def edit_distance(string1,string2):
    if len(string1) == 0:return len(string2)
    if len(string2) == 0:return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1],string2) + 1,'DEL {}'.format(tail_s1)),
        (edit_distance(string1,string2[:-1]) + 1,'ADD {}'.format(tail_s2))
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1],string2[:-1]) + 0,'')
    else:
        both_forward = (edit_distance(string1[:-1],string2[:-1]) + 1,
                        'SUB {} => {}'.format(tail_s1,tail_s2))
    candidates.append(both_forward)
    
    min_distance, operation = min(candidates,key=lambda x:x[0])
    solution[(string1,string2)] = operation
    
    return min_distance

In [17]:
edit_distance('ABCDE','ABCCEFD')

3

In [18]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('A', 'ABCCEFD'): 'ADD D',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('AB', 'ABCCEFD'): 'ADD D',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABC', 'ABCCEFD'): 'ADD D',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCD', 'ABCCEFD'): '',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'ABCCEFD'): 'ADD D'}

### 作业一：解析编辑距离的结果

In [19]:
"""
递归终止条件：两字符串相等，返回[]
如果字符串不相等，却无需操作，那么就是长度相等且最后一个字符相等的情况，直接判断前n-1个字符的操作；
如果字符串不相等，而长度相等，那么替换，然后判断前n-1字符的操作；
如果字符串不相等，而后一个字符串更长，那么前一字符串插入，然后判断前n-1字符的操作；
如果字符串不相等，而前一个字符串更长，那么前一字符串删除，然后判断前n-1字符的操作。
"""

def parse_solution(string1,string2):
    '''
    :string1,string2: 两个字符串
    :return: 包含插入、删除或替换三种操作的列表
    '''
    if string1 == string2:return []
    
    operations = []
    operation = solution[(string1,string2)]
    
    if operation is '':
        operations = parse_solution(string1[:-1],string2[:-1]) + operations
    else:
        operations.append(operation)
        if len(string1) == len(string2):
            operations = parse_solution(string1[:-1],string2[:-1]) + operations
        elif len(string1) < len(string2):
            operations = parse_solution(string1,string2[:-1]) + operations
        elif len(string1) > len(string2):
            operations = parse_solution(string1[:-1],string2) + operations
            
    return operations

In [20]:
parse_solution('ABCDE','ABCCEFD')

['SUB D => C', 'ADD F', 'ADD D']

In [21]:
'''换成前一个字符串更长的例子'''
print(edit_distance('ABCCEFD','ABCDE'))
print(parse_solution('ABCCEFD','ABCDE'))

3
['SUB C => D', 'DEL F', 'DEL D']


In [22]:
chinese_charaters = open('article_9k.txt').read()

In [23]:
chinese_charaters[:100]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体验版内测稳定版暂不受影响以确保工程师可以集中全部精力进行系统优化工作有人猜测这也是将精力主要用到MIUI9的研发之中MI'

In [24]:
import pinyin

In [25]:
pinyin.get('你好',format='strip',delimiter=' ')

'ni hao'

In [26]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format="strip", delimiter=" ")

In [27]:
chinese_pinyin_copys = chinese_to_pinyin(chinese_charaters)

In [28]:
chinese_pinyin_copys[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [29]:
len(chinese_pinyin_copys)

129433034

In [30]:
import re

In [31]:
def tokens(text):
    '''找出所有的拼音'''
    return re.findall('[a-z]+',text.lower())

In [32]:
' '.join(tokens(chinese_pinyin_copys[:100]))

'ci wai zi ben zhou yue ri qi chu xiao mi shou ji deng kuan ji xing wai qi yu ji xing yi'

In [33]:
from collections import Counter,defaultdict

In [34]:
pinyin_count = Counter(tokens(chinese_pinyin_copys))

In [35]:
def correct(word):
    '''基于编辑距离，找出所有可能的拼音
    偏向于编辑距离为0,1,2的，否则返回自身的拼音。
    '''
    candidates = (konwn(edits0(word)) or
                  konwn(edits1(word)) or
                  konwn(edits2(word)) or
                  [word]
                 )
    return max(candidates,key=pinyin_count.get)

In [36]:
def konwn(words):
    ''''''
    return {w for w in words if w in pinyin_count}

def edits0(word):
    '''编辑距离为0，也就是它本身'''
    return {word}

def edits2(word):
    '''
    返回与目标拼音的编辑距离为2的所有拼音
    '''
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [37]:
def edits1(word):
    '''返回与这个拼音只有一个编辑距离的所有拼音
    '''
    pairs      = splits(word)
    deletes    = [a + b[1:]               for (a,b) in pairs if b]
    transposes = [a + b[1] + b[0] + b[2:] for (a,b) in pairs if len(b) > 1]
    replaces   = [a + c + b[1:]           for (a,b) in pairs for c in alphabet if b]
    inserts    = [a + c + b               for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)
    
def splits(word):
    '''
    '''
    return [(word[:i],word[i:]) for i in range(len(word)+1)]

In [38]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [39]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [40]:
print(edits0('pinyin'))

{'pinyin'}


In [41]:
print(edits1('pinyin'))

{'pinpyin', 'pinyinj', 'piyyin', 'cpinyin', 'pbnyin', 'poinyin', 'ppinyin', 'pqnyin', 'pinyan', 'pinyyin', 'pjnyin', 'kinyin', 'pinyix', 'piyin', 'ainyin', 'gpinyin', 'pionyin', 'pimnyin', 'pinyil', 'inyin', 'pinyjn', 'fpinyin', 'pjinyin', 'pinyibn', 'pitnyin', 'ponyin', 'minyin', 'pinxin', 'ypinyin', 'oinyin', 'pinyyn', 'hinyin', 'piknyin', 'pifnyin', 'piunyin', 'pibyin', 'pinyitn', 'pinyion', 'qpinyin', 'pinyrn', 'piuyin', 'pinjyin', 'pinyein', 'pinyinm', 'pipyin', 'pinyik', 'piayin', 'npinyin', 'pminyin', 'pinyiwn', 'pienyin', 'pinycn', 'pinyinc', 'pinwin', 'pinyinb', 'pinnyin', 'pinymin', 'pinyifn', 'pinzyin', 'pinyvin', 'pinyine', 'piqnyin', 'pinyina', 'pindyin', 'pinyiln', 'ppnyin', 'piniin', 'pisyin', 'pinyie', 'pinuin', 'pinwyin', 'pinyn', 'pinyidn', 'pinynin', 'pinyini', 'rpinyin', 'pinyxn', 'pinyi', 'prnyin', 'dpinyin', 'hpinyin', 'pinyien', 'pinyinw', 'pinhin', 'pingyin', 'vinyin', 'pivnyin', 'pintyin', 'pinyjin', 'pinyiv', 'psnyin', 'pbinyin', 'pnyin', 'pinyzin', 'pijyin', 

In [42]:
len(edits1('pinyin'))

338

In [43]:
len(edits2('pinyin'))

52168

In [44]:
correct('yin')

'yin'

In [45]:
correct('yign')

'ying'

In [46]:
correct('yinn')

'ying'

### 文字:这是一个测试

In [48]:
def correct_sequence_pinyin(text_pingyin):
    return ' '.join(map(correct, text_pingyin.split()))

In [49]:
correct_sequence_pinyin('zhe sih yi ge ce sho')

'zhe shi yi ge ce shi'

### 文字：我想上清华大学

In [50]:
correct_sequence_pinyin('wo xiang shagn qinng hua da xue')

'wo xiang shang qing hua da xue'

## 思考题：如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？

## 提示：用语言模型。

In [51]:
sentence_pinyin = 'woxiangshagnqinnghuadaxue'

In [ ]:
方案1：动态规划，先把前面的处理好，再不断往后纠正

对于一个短句，划分可能的句子；
对于每一个句子，用编辑距离进行修正；
再用语言模型计算每一个句子的概率；
用两种算法，gready search 和 beam search，去得到概率最高的句子；
动态规划，往后继续计算。

In [52]:
pinyin_corpus = tokens(chinese_pinyin_copys)

In [53]:
pinyin_corpus_bi = [''.join(pinyin_corpus[i:i+2]) for i in range(len(pinyin_corpus[:-2]))]

In [54]:
pinyin_corpus[:5]

['ci', 'wai', 'zi', 'ben', 'zhou']

In [55]:
pinyin_corpus_bi[:5]

['ciwai', 'waizi', 'ziben', 'benzhou', 'zhouyue']

In [56]:
pinyin_count_bi = Counter(pinyin_corpus_bi)

In [57]:
def unigram(py):
    if py in pinyin_corpus:
        return (pinyin_count[py] + 1) / (len(pinyin_count)+len(pinyin_corpus))
    else:
        return 1 / (len(pinyin_count)+len(pinyin_corpus))


def bigram(py):
    _,py1 = py.split()
    if py in pinyin_count_bi:
        prob = (pinyin_count_bi[py] + 1) / (pinyin_count_bi[py1]+len(pinyin_corpus))
        return prob
    else:
        return 1 / (pinyin_count[py1]+len(pinyin_corpus))
    
def calcu_prob(sentence):
    sentences = sentence.split()
    sentence_prob = 1
    for i,py in enumerate(sentences[:-1]):
        next_py = sentences[i+1]
        sentence_prob *= bigram(py,next_py)
    return sentence_prob

## 原谅我写不出来了，只能参考优秀作业了。。。